<a href="https://colab.research.google.com/github/Ratchadakorn26/GA_project/blob/main/Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

# ค่าคงที่
CYCLE_TIME = 89  # รอบเวลาทั้งหมด
q_values = [1802, 1497, 542]  # อัตราการไหลของรถแต่ละเฟส (คัน/ชั่วโมง)
x_values = [0.263, 0.233, 0.17]  # Degree of Saturation
POP_SIZE = 10  # ขนาดประชากร
MUTATION_RATE = 0.1  # อัตราการกลายพันธุ์
MAX_GENERATIONS = 1000  # จำนวนรอบสูงสุด

# ฟังก์ชันคำนวณค่าเฉลี่ยเวลารอคอยของทั้ง 3 เฟส
def calculate_wait_time(green_times):
    total_wait_time = 0
    for i in range(3):
        g_i = green_times[i]
        q_i = q_values[i]
        x_i = x_values[i]

        term1 = CYCLE_TIME * ((1 - (g_i / CYCLE_TIME)) ** 2) / (1 - (g_i / CYCLE_TIME) * x_i)
        term2 = (x_i ** 2) / (2 * q_i * (1 - x_i))
        term3 = ((CYCLE_TIME / q_i) ** 0.5) * (x_i ** (2 + (5 * g_i / CYCLE_TIME)))

        d_i = term1 + term2 - term3
        total_wait_time += d_i

    return total_wait_time / 3  # ค่าเฉลี่ยของทั้ง 3 เฟส

# ฟังก์ชัน Fitness (ค่าฟิตเนส = ค่าเฉลี่ยรอคอยของทั้ง 3 เฟส)
def fitness_function(green_times):
    return calculate_wait_time(green_times)

# 1️⃣ สร้างประชากรเริ่มต้น
def initialize_population():
    population = []
    for _ in range(POP_SIZE):
        proportions = np.array(q_values) / sum(q_values)
        green_times = (proportions * CYCLE_TIME).astype(int)
        green_times[-1] = CYCLE_TIME - green_times[:-1].sum()  # ปรับให้ผลรวมเป็น 89
        population.append(green_times)
    return np.array(population)

# 2️⃣ เลือกโครโมโซมที่มีค่าเฉลี่ยต่ำสุด 5 โครโมโซม
def select_best(population):
    fitness_scores = np.array([fitness_function(ind) for ind in population])
    sorted_indices = np.argsort(fitness_scores)[:5]  # 5 โครโมโซมที่ดีที่สุด
    return population[sorted_indices]

# 3️⃣ Crossover (จับคู่พ่อแม่แล้วทำการเปลี่ยนยีน)
def crossover(parents):
    new_population = []
    for _ in range(POP_SIZE // 2):
        parent1, parent2 = random.sample(list(parents), 2)
        child1 = np.copy(parent1)
        child2 = np.copy(parent2)
        crossover_point = random.randint(1, 2)
        child1[:crossover_point], child2[:crossover_point] = child2[:crossover_point], child1[:crossover_point]

        # ปรับให้ค่ารวมไม่เกิน 89
        child1 = np.round((child1 / child1.sum()) * CYCLE_TIME).astype(int)
        child1[-1] = CYCLE_TIME - child1[:-1].sum()
        child2 = np.round((child2 / child2.sum()) * CYCLE_TIME).astype(int)
        child2[-1] = CYCLE_TIME - child2[:-1].sum()

        new_population.extend([child1, child2])
    return np.array(new_population)

# 4️⃣ Mutation (กลายพันธุ์บางตำแหน่ง)
def mutate(population):
    for i in range(len(population)):
        if random.random() < MUTATION_RATE:
            mutation_idx = np.random.randint(3)
            change = np.random.randint(-2, 3)
            population[i][mutation_idx] = max(5, min(population[i][mutation_idx] + change, CYCLE_TIME - 10))

            # ปรับให้ค่ารวมไม่เกิน 89
            population[i] = np.round((population[i] / population[i].sum()) * CYCLE_TIME).astype(int)
            population[i][-1] = CYCLE_TIME - population[i][:-1].sum()
    return population

# 5️⃣ Genetic Algorithm (วนซ้ำจนได้ค่าที่ดีที่สุด)
def genetic_algorithm():
    population = initialize_population()
    best_wait_time = float('inf')
    best_solution = None
    repeated_count = 0
    prev_best_wait_time = None

    for generation in range(MAX_GENERATIONS):
        selected_parents = select_best(population)
        new_population = crossover(selected_parents)
        new_population = mutate(new_population)

        # อัปเดตค่าที่ดีที่สุด
        fitness_scores = np.array([fitness_function(ind) for ind in new_population])
        best_idx = np.argmin(fitness_scores)
        best_current_wait_time = fitness_scores[best_idx]
        best_current_solution = new_population[best_idx]

        print(f"Generation {generation}: Green Times = {best_current_solution}, Avg Wait Time = {best_current_wait_time:.6f}")

        if best_current_wait_time < best_wait_time:
            best_wait_time = best_current_wait_time
            best_solution = best_current_solution
            repeated_count = 0
        else:
            repeated_count += 1

        # หยุดถ้ามีค่าที่ดีที่สุดซ้ำกัน 50 รอบ
        if repeated_count >= 50:
            break

        population = new_population

    print("\n✅ Optimal Green Times Found:")
    print(f"🔹 Green Times: {best_solution}")
    print(f"🔹 Avg Wait Time: {best_wait_time:.6f}")
    return best_solution

# เรียกใช้งาน Genetic Algorithm
best_green_times = genetic_algorithm()


Generation 0: Green Times = [41 34 14], Avg Wait Time = 43.900241
Generation 1: Green Times = [41 34 14], Avg Wait Time = 43.900241
Generation 2: Green Times = [40 35 14], Avg Wait Time = 43.865758
Generation 3: Green Times = [40 35 14], Avg Wait Time = 43.865758
Generation 4: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 5: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 6: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 7: Green Times = [39 34 16], Avg Wait Time = 43.606229
Generation 8: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 9: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 10: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 11: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 12: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 13: Green Times = [40 34 15], Avg Wait Time = 43.747458
Generation 14: Green Times = [40 34 15], Avg Wait Time = 43.747458
Gener